# Supervised learning - reusable data creation

The notebook is intended to serve as a guide on how to prepare and manipulate data for reusability, and avoid the need to re-run the whole notebook.

We will demonstrate how we create the HDF files for use in File 'Feature_Importance_and_Feature_Ablation.ipynb' in the first section, and the creation of HDF files for use in 'Analysis and Failure Analysis.ipynb' and 'Learning curve for all supervised models.ipynb' in the second section below.



In [1]:
# Built-in libraries
import re
import string
import warnings

# Third-party libraries for data handling and processing
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from tqdm import tqdm

# Feature extraction
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import KeyedVectors

# Pre-processing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler, MaxAbsScaler
from imblearn.over_sampling import SMOTE, SMOTENC


# Miscellaneous
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
tqdm.pandas()
warnings.filterwarnings("ignore")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 0.  
#### data_X.csv and y.csv below are clean and manipulated data created in file 'supervised_1007.ipynb'.  
#### data_X.csv and y.csv are used directly in file 'Feature Importance and Feature Ablation before SMOTE.ipynb'

In [3]:
X = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_X.csv')
y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/y.csv')


In [4]:
print(X.shape)
print(y.shape)

(164034, 8435)
(164034, 1)


### 1.    
#### Creating Preprocessed data X_smote and y_smote used in file 'Feature_Importance_and_Feature_Ablation.ipynb' by conducting SMOTE on both data_X.csv and y.csv

In [5]:
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

In [6]:
# shape
print(X_smote.shape, y_smote.shape)

(256454, 8435) (256454, 1)


In [7]:
X_smote.to_hdf('/content/drive/MyDrive/Colab Notebooks/X_smote.h5', 'X')
y_smote.to_hdf('/content/drive/MyDrive/Colab Notebooks/y_smote.h5', 'y')

### 2.  
#### Creating Preprocessed data X.h5 and y.h5 used in file 'Analysis and Failure Analysis.ipynb' and 'Learning curve for all supervised models.ipynb'

In [8]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/complaints.csv')
df.shape

(4028530, 18)

In [9]:
# preprocess the narrative column
def preprocess_narrative(text):
    # Lowercase
    text = text.lower()

    # Remove XXXX like pattern
    text = re.sub(r'x{2,}', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove stopwords
    text = " ".join([word for word in text.split() if word not in stop_words])

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text)

    # Remove leading and trailing spaces
    text = text.strip()

    return text


# define a function get mean word2vec vector for a narrative
def get_mean_word2vec(narrative):

    # initialize vector
    vector = np.zeros(300)

    # get all words in narrative
    words = narrative.split()
    num_words = len(words)

    if num_words == 0:  # edge case: empty narrative
        return vector

    # calculate word vectors using list comprehension
    word_vectors = [word2vec[word] for word in words if word in word2vec]

    if word_vectors:
        vector = np.mean(word_vectors, axis=0)

    return vector

In [10]:
# drop rows with missing values on narrative column and dispute column
df.dropna(subset=['Consumer complaint narrative', 'Consumer disputed?'], inplace=True)

In [11]:
# preprocess the narrative column
df['narrative_clean'] = df['Consumer complaint narrative'].progress_apply(preprocess_narrative)


100%|██████████| 164034/164034 [00:16<00:00, 9687.04it/s] 


In [12]:
# encode target variable
df['Consumer disputed?'] = df['Consumer disputed?'].map({'Yes': 1, 'No': 0})

y_tw = df['Consumer disputed?'].values

Feature Engineering: days between date received and date sent to company

In [13]:
# feature engineering: days between date received and date sent to company

# convert date received and date sent to company to datetime
df['Date received'] = pd.to_datetime(df['Date received'])
df['Date sent to company'] = pd.to_datetime(df['Date sent to company'])

# calculate days between date received and date sent to company
df['days_between'] = (df['Date sent to company'] - df['Date received']).dt.days

# int
df['days_between'] = df['days_between'].astype(int)

# fill with 0 for any negative values
df['days_between'] = df['days_between'].apply(lambda x: 0 if x < 0 else x)

Feature Engineering: number of words in complaint narrative

In [14]:
# feature engineering: number of words in narrative (on original narrative column)

# get number of words in narrative

df['narrative_word_count'] = df['Consumer complaint narrative'].apply(lambda x: len(x.split()))

Feature Engineering: number of disputed for the company in the last 90 days

In [15]:
# feature engineering:
# For each row, calculate the number of disputed for that company in the last 90 days
# Define the lambda expression to compute disputed count for each row

df["Disputed_count_last_90_days"] = df.progress_apply(
    lambda row: df[
        (df["Date received"] <= row["Date received"]) &
        (df["Date received"] > (row["Date received"] - pd.Timedelta(days=90))) &
        (df["Company"] == row["Company"]) &
        (df["Consumer disputed?"] == 1)
    ].shape[0],
    axis=1
)

100%|██████████| 164034/164034 [36:03<00:00, 75.81it/s]


Feature Engineering: number of complaints for the company in the last 90 days

In [16]:
# feature engineering:
# For each row, calculate the number of complaints for that company in the last 90 days
df["Complaints_last_90_days"] = df.progress_apply(
    lambda row: df[
        (df["Date received"] <= row["Date received"]) &
        (df["Date received"] > (row["Date received"] - pd.Timedelta(days=90))) &
        (df["Company"] == row["Company"])
    ].shape[0],
    axis=1
)

100%|██████████| 164034/164034 [37:24<00:00, 73.07it/s]


Feature Engineering: ratio. Rate of disputed

In [17]:
# feature engineering:
# ratio of number of disputed in the last 90 days / number of complaints in the last 90 days

df['Disputed_ratio_last_90_days'] = df['Disputed_count_last_90_days'] / df['Complaints_last_90_days']

In [18]:
# import word2vec model (this take a while to load)
word2vec = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Colab Notebooks/GoogleNews-vectors-negative300.bin', binary=True)

In [21]:
# compute mean tfidf weighted word2vec vector for each narrative (take a while)
df['narrative_clean_'] = df['narrative_clean'].progress_apply(get_mean_word2vec)

# convert df['narrative_clean']  to numpy array tw_array
tw_array = np.array(df.narrative_clean_.tolist())


100%|██████████| 164034/164034 [00:46<00:00, 3520.42it/s]


Drop columns logic

In [22]:
# drop columns
# 1. date received (computed to days between, redundant)
# 2. date sent to company (computed to days between, redundant)
# 3. consumer complaint narrative (already vectorized to tfidf features)
# 4. complaint ID (unique identifier, not useful for modeling)
# 5. timely response? (highly skewed, 99-1, not useful for modeling
# 6. consumer consent provided? (only one value, no variance not useful for modeling)
# 7. submitted via (only one value, no variance not useful for modeling)
# 8. narrative_clean (already vectorized to tfidf features)
# 9. zip code (high cardinality, would expand almost 7000 dimension if included. likely correlate with State)
# 10. consumer disputed? (remove target variable)

drop_cols = [
    'Date received',
    'Date sent to company',
    'Consumer complaint narrative',
    'Complaint ID',
    'Timely response?',
    'Consumer consent provided?',
    'Submitted via',
    'narrative_clean',
    'ZIP code',
    'Consumer disputed?',
    'narrative_clean_'
]

df.drop(drop_cols, axis=1, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164034 entries, 31338 to 4028159
Data columns (total 15 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Product                       164034 non-null  object 
 1   Sub-product                   111863 non-null  object 
 2   Issue                         164034 non-null  object 
 3   Sub-issue                     83055 non-null   object 
 4   Company public response       78122 non-null   object 
 5   Company                       164034 non-null  object 
 6   State                         163575 non-null  object 
 7   Tags                          26994 non-null   object 
 8   Company response to consumer  164034 non-null  object 
 9   days_between                  164034 non-null  int64  
 10  narrative_word_count          164034 non-null  int64  
 11  Disputed_count_last_90_days   164034 non-null  int64  
 12  Complaints_last_90_days       164034 no

In [23]:
drop_cols = [
    'narrative_clean_'
]

df.drop(drop_cols, axis=1, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164034 entries, 31338 to 4028159
Data columns (total 14 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Product                       164034 non-null  object 
 1   Sub-product                   111863 non-null  object 
 2   Issue                         164034 non-null  object 
 3   Sub-issue                     83055 non-null   object 
 4   Company public response       78122 non-null   object 
 5   Company                       164034 non-null  object 
 6   State                         163575 non-null  object 
 7   Tags                          26994 non-null   object 
 8   Company response to consumer  164034 non-null  object 
 9   days_between                  164034 non-null  int64  
 10  narrative_word_count          164034 non-null  int64  
 11  Disputed_count_last_90_days   164034 non-null  int64  
 12  Complaints_last_90_days       164034 no

In [24]:
# fill na with 'Unknown'
df.fillna('Unknown', inplace=True)

One hot encode categorical features

In [25]:
# one hot encode categorical features
df = pd.get_dummies(df, drop_first=True)

Concatenate word2vec features and encoded dataframe

In [26]:
# concat df with X_bow and df.values
X_tw = np.concatenate((tw_array, df.values), axis=1)

Imbalance Handling

In [27]:
# smote to handle class imbalance
smote = SMOTE(random_state=42)
X_tw_smote, y_tw_smote = smote.fit_resample(X_tw, y_tw)

# shape
print(X_tw_smote.shape, y_tw_smote.shape)

(256454, 3735) (256454,)


Exporting cleaned and manipulated data to csv for reusability and avoid re-run the whole notebook.

In [40]:
# Exporting cleaned and manipulated data to hdf
# save X and y for future use.
# (ready-to-train data and target)

# Define the path where we want to save the hdf files
X_tw_smote_path = '/content/drive/MyDrive/Colab Notebooks/X.h5'
y_tw_smote_path = '/content/drive/MyDrive/Colab Notebooks/y.h5'

# Create a DataFrame
data_X = pd.DataFrame(X_tw_smote)

# Save data_X to HDF
data_X.to_hdf(X_tw_smote_path, key='123')

# Save y to HDF
pd.DataFrame(y_tw_smote).to_hdf(y_tw_smote_path, key='124')

In [ ]:
# to restore, do the following:

# data_X = pd.read_hdf('X.h5', key='123')
# y = pd.read_csv('y.h5.csv', key='124')